### Fetch response

In [1]:
import pandas as pd

test_df = pd.read_parquet("./data/66-test-dataset-with-adrd.parquet")

test_df.head()

,Question,Context,Answer,adrd_qwen3_14b,adrd_dpv3,gpt-o3
0,Can you recommend activities that are suitable...,Content: What are some tips for Finding Activi...,When looking for activities for someone with d...,Engaging a loved one with dementia in meaningf...,I understand how important it is to find meani...,I understand that finding the right activities...
1,What are the benefits of hospice care for indi...,Content: How to be an effective caregiver in t...,Hospice care offers several benefits for indiv...,Caring for someone with advanced Alzheimer's i...,Hospice care can provide immense comfort and s...,I understand that navigating the complexities ...
2,How can I navigate difficult family dynamics w...,Content: How to Handle Seemingly Manipulative ...,When navigating difficult family dynamics whil...,Navigating family dynamics while caring for a ...,Navigating difficult family dynamics while car...,I understand how overwhelming it can feel to j...
3,What are the early signs of dementia?,Content: None The early warning signs of demen...,The early signs of dementia can vary from pers...,It's completely understandable to feel concern...,I understand how important it is to recognize ...,I understand how overwhelming it can be to not...
4,"How does palliative care differ from hospice, ...",Content: What end-of-life care options are ava...,Palliative care and hospice care are both form...,Understanding the distinction between palliati...,Palliative care and hospice care both aim to i...,I understand that navigating the differences b...


In [2]:
question_array = test_df.Question.to_list()

print(question_array)

['Can you recommend activities that are suitable for someone with dementia to engage in and enjoy?', "What are the benefits of hospice care for individuals in the advanced stages of Alzheimer's disease?", "How can I navigate difficult family dynamics when caring for a loved one with Alzheimer's?", 'What are the early signs of dementia?', 'How does palliative care differ from hospice, and how can it help improve the quality of life for someone with advanced dementia?', "How can I ensure my loved one's safety at home, especially if they have a tendency to wander or become agitated?", 'What activities are suitable for people with dementia, and how can I encourage participation?', 'Are there any risk factors that can contribute to developing delirium?', 'Are certain types of anesthesia more likely to cause delirium?', "How do I navigate legal issues related to caring for a loved one with Alzheimer's, such as power of attorney and guardianship?", "How do I obtain legal guardianship or power

#### CaLM-ADRD

In [ ]:
from test.evaluation import call_api_batch

answer_array = await call_api_batch(question_array)

print(answer_array)

2025-07-17 @ 16:01:26 | INFO     | detect_intention | User's query: Can you recommend activities that are suitable for someone with dementia to engage in and enjoy?
2025-07-17 @ 16:01:26 | INFO     | detect_intention | User's query: What are the benefits of hospice care for individuals in the advanced stages of Alzheimer's disease?
2025-07-17 @ 16:01:26 | INFO     | adaptive_rag_decision | Adaptive decision | Can you recommend activities that are suitable for someone with dementia to engage in and enjoy? | 
2025-07-17 @ 16:01:26 | INFO     | detect_intention | User's query: How can I navigate difficult family dynamics when caring for a loved one with Alzheimer's?
2025-07-17 @ 16:01:26 | INFO     | detect_intention | User's query: What are the early signs of dementia?
2025-07-17 @ 16:01:26 | INFO     | adaptive_rag_decision | Adaptive decision | What are the benefits of hospice care for individuals in the advanced stages of Alzheimer's disease? | 
2025-07-17 @ 16:01:26 | INFO     | dete

CancelledError: 

2025-07-17 @ 16:01:40 | SUCCESS  | retrieve_documents | Similarity search retrieved | 3 | documents
2025-07-17 @ 16:01:40 | SUCCESS  | retrieve_documents | Similarity search retrieved | 3 | documents
2025-07-17 @ 16:01:40 | SUCCESS  | retrieve_documents | Similarity search retrieved | 3 | documents
2025-07-17 @ 16:01:40 | SUCCESS  | retrieve_documents | Similarity search retrieved | 3 | documents
2025-07-17 @ 16:01:40 | SUCCESS  | retrieve_documents | Similarity search retrieved | 3 | documents
2025-07-17 @ 16:01:40 | SUCCESS  | retrieve_documents | Similarity search retrieved | 3 | documents


2025-07-17 @ 16:01:40 | SUCCESS  | retrieve_documents | Similarity search retrieved | 3 | documents
2025-07-17 @ 16:01:40 | SUCCESS  | retrieve_documents | Similarity search retrieved | 3 | documents
2025-07-17 @ 16:01:40 | SUCCESS  | retrieve_documents | Similarity search retrieved | 3 | documents
2025-07-17 @ 16:01:40 | SUCCESS  | retrieve_documents | Similarity search retrieved | 3 | documents
2025-07-17 @ 16:01:40 | SUCCESS  | retrieve_documents | Similarity search retrieved | 3 | documents
2025-07-17 @ 16:01:40 | SUCCESS  | retrieve_documents | Similarity search retrieved | 3 | documents
2025-07-17 @ 16:01:40 | SUCCESS  | retrieve_documents | Similarity search retrieved | 3 | documents
2025-07-17 @ 16:01:40 | SUCCESS  | retrieve_documents | Similarity search retrieved | 3 | documents
2025-07-17 @ 16:01:40 | SUCCESS  | retrieve_documents | Similarity search retrieved | 3 | documents
2025-07-17 @ 16:01:40 | SUCCESS  | adaptive_rag_decision | Adaptive decision | True | for extra retr

In [4]:
test_df.loc[:, "adrd_qwen3_14b"] = answer_array

test_df.to_parquet("./data/66-test-dataset-with-adrd.parquet")

In [ ]:
import pandas as pd

eval_df = pd.read_parquet("./data/66-test-dataset-with-adrd.parquet")

eval_df.head()

#### From OpenAI GPT

In [ ]:
from dotenv import load_dotenv
from langchain.chat_models import init_chat_model
from langchain_core.prompts import PromptTemplate

load_dotenv()

o3 = init_chat_model("o3-mini", model_provider="openai", temperature=1)


prompt = PromptTemplate(
    input_variables=["question"],
    template="""
You are a compassionate healthcare consultant specializing in caregiving for Alzheimer's Disease and Related Dementias
(ADRD). Your job is to provide empathetic, knowledgeable, and structured support to caregivers facing emotional,
practical, and medical challenges. You answer questions based on the provided context (user's input and chat history)
while offering responses that are warm, informative, and actionable.


# Thinking Process

Think step by step, but only keep a minimum draft for each thinking step, with 5 words at most.

# Approach
	•	Answer starts with a paragraph (2-3 sentences) of introduction and emotional recognition and support, then answer the question in a structured way, and ends with a paragraph (2-3 sentences) of conclusion.
	•	Warm yet professional: Speak with kindness, avoiding overly clinical or detached language.
	•	Non-judgmental & supportive: Caregiving is challenging—reassure the user that they are doing their best.
	•	Use proper in text citations to reference the sources and support your statements. In format of "[<index>]" (e.g. "[1]", "[2]", "[3]", etc.).

# Context

The caregiver's current query:
{question}
""",
)

chain = prompt | o3

chain.invoke({"question": "Hello, how are you?"})

In [ ]:
from langchain_core.messages import BaseMessage

res_list = []

for idx, question in enumerate(question_array):
    response: BaseMessage = chain.invoke({"question": question})

    res_list.append(response.content)
    print(f"Processed {idx} questions")

print(res_list[:5])

In [6]:
test_df.loc[:, "gpt-o3"] = res_list

test_df.to_parquet("./data/66-test-dataset-with-adrd.parquet")